In [6]:
# ['T1','TH','EA','EL','PI','PR','PG','SF','SR','SA']
import torch


In [7]:
model_path = '/Users/monaabd/Desktop/EmotiBit Final Models/3S_32H_3L/model29.pt'
model = torch.load(model_path)
input_size = model.input_size
val_loader = torch.load('/Users/monaabd/Desktop/3 sec dataloader/val3_dataloader.pt')

ModuleNotFoundError: No module named 'model'

In [ ]:
model.eval()  # Set the model to evaluation mode

importances = torch.zeros(input_size)  # input_size is the number of input features
noise_or_value = 0.025
for i, (inputs, targets) in enumerate(val_loader):
    original_output = model(inputs)
    for j in range(input_size):
        perturbed_input = inputs.clone()
        # Add random noise or a constant value to the j-th feature across all timesteps and batch items
        perturbed_input[:, :, j] += noise_or_value
        perturbed_output = model(perturbed_input)
        # Measure the change in the output
        change = torch.abs(perturbed_output - original_output)
        importances[j] += change.mean().item()

# Normalize the importances by the number of validation samples
importances /= len(val_loader)